In [100]:
import pandas as pd
import numpy as np
import math
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
from time import sleep
from tqdm import trange
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
import math as mt
from numpy.linalg import inv, det
from matplotlib import pyplot as plt
%matplotlib inline

## Read data

In [101]:
mfeat = pd.read_csv("mfeat/mfeat-fac", delimiter="\0", sep="\0", header=None )
mfeatFou = pd.read_csv("mfeat/mfeat-fou", delimiter="\0", sep="\0", header=None )
mfeatka = pd.read_csv("mfeat/mfeat-kar", delimiter="\0", sep="\0", header=None )

y = [0 for x in range(0, 200)]
y += [1 for x in range(200, 400)]
y += [2 for x in range(400, 600)]
y += [3 for x in range(600, 800)]
y += [4 for x in range(800, 1000)]
y += [5 for x in range(1000, 1200)]
y += [6 for x in range(1200, 1400)]
y += [7 for x in range(1400, 1600)]
y += [8 for x in range(1600, 1800)]
y += [9 for x in range(1800, 2000)]

In [4]:
y.count(9)

200

mfeat

In [5]:
mfeat.values[:1]

array([['   98  236  531  673  607  647    2    9    3    6    8    5  225  517  652  624  628  994    7   22   28   13   10   19  305  481  667  663 1009  727   38   28   18   11   20   10  287  567  651  742  824  900   26   34   30    8   16   13  248  556  631  796  926  748   39   34   18    9   17   12  248  540  506  814 1051  728   38   28    5   13   16    8  246  518  751  579  699 1062   13   30   28   10   16   16  276  344  682  500  709  916   10   30   23   17   15   14  357  435  829  610  745  994   20    7   24   12   10    9  355  409  477  886  976  723   30   24   14    7    7    8  290  352  435  753  894  751   29   29    2   13   13   14  260  286  562  698  665  757   11    8   15   14    9    9  238  292  586  698  733  707    9    6   15   11   10   16  294  406  654  644  741 1000   18   23   17    9   20   11  302  418  561  709  961  776   21   25   12   12   19   10  360  328  607  984 1186  599   29    7   14    6    9    9  362  314  924  733  601 1216 

In [6]:
mfeatFou.values[:1]

array([['  0.06588172  0.19731169  0.10382563  0.27036171  0.61607757  0.03585575  0.42457192  0.08970119  0.36777339  0.03706516  0.07594178  0.26390595  0.07790313  0.37200891  0.21948485  0.05532691  0.10890633  0.12775213  0.11405136  0.01858087  0.10262683  0.17268198  0.07303320  0.14034402  0.13219889  0.07601515  0.03251728  0.12883987  0.12052641  0.01661452  0.10258255  0.10162490  0.06572986  0.03521252  0.07900934  0.08377258  0.02149083  0.12162476  0.06751158  0.03377662  0.09901323  0.05940796  0.08128562  0.05234493  0.09049952  0.06358191  0.08110426  0.05910051  0.09723591  0.04157573  0.08351457  0.04237858  0.08296503  0.07717036  0.11673209  0.03476068  0.10948465  0.01642286  0.09935212  0.11376267  0.08637167  0.16765423  0.10628061  0.12578294  0.09067476  0.08125645  0.12344814  0.11337439  0.07434256  0.28166630  0.06779520  0.34418162  0.03896293  0.39436572  0.04997072  0.34487096']],
      dtype=object)

### Data preprocessing

In [105]:
#Retirando os espaços. Transformando da elemento em uma lista com valores númericos, Atribuindo isso a uma 
# lista com tudo processado e normaliza
def preprocessing(data):
    new_data = []
    for ex in data:
        ex = ex[0].split(" ")
        exemplos1 = []
        for element in ex:
            if element != "":
                exemplos1.append(element)
        new_data.append(exemplos1) 
    
    ## Normaliza
    new_data = np.array(new_data).astype(float)
    scaler = StandardScaler()
    scaler.fit(new_data)
    new_data = scaler.transform(new_data)
    return new_data
   


In [106]:
mfeat =  preprocessing(mfeat.values)
mfeatFou =  preprocessing(mfeatFou.values)
mfeatKa =  preprocessing(mfeatka.values)

In [107]:
print(mfeat.shape)
print(mfeatFou.shape)
print(mfeatKa.shape)

(2000, 216)
(2000, 76)
(2000, 64)


# Dataset (Train and test)

In [56]:
X_train_mfeat, X_test_mfeat, y_train_mfeat, y_test_mefeat =  train_test_split(mfeat,y, test_size=0.10, random_state=42)
X_train_mfeatFou, X_test_mfeatFou, y_train_mfeatFou, y_test_mefeatFou =  train_test_split(mfeatFou,y, test_size=0.10, random_state=42)
X_train_mfeatKa, X_test_mfeatKa, y_train_mfeatKa, y_test_mefeatKa =  train_test_split(mfeatKa,y, test_size=0.10, random_state=42)

In [11]:
print(X_test_mfeat.shape)
print(X_test_mfeatFou.shape)
print(X_test_mfeatKa.shape)

(200, 216)
(200, 76)
(200, 64)


# Calculates the Euclidean matrices

In [108]:
from scipy.spatial.distance import pdist, squareform

matrix_euclian_mfeat = squareform(pdist(mfeat, metric='euclidean'))
matrix_euclian_mfeatFou = squareform(pdist(mfeatFou, metric='euclidean'))
matrix_euclian_mfeatKa = squareform(pdist(mfeatKa, metric='euclidean'))



In [109]:
matrix_euclian_mfeat.shape

(2000, 2000)

In [110]:
print(matrix_euclian_mfeat.shape)
print(matrix_euclian_mfeatFou.shape)
print(matrix_euclian_mfeatKa.shape)

(2000, 2000)
(2000, 2000)
(2000, 2000)


# Bayes

In [111]:

def calc_density(x, priori, mean, sigma, classes):
    apriori = priori
    mean = mean
    sigma = sigma
    classes = classes
    x = x
    densities = []

    for c in range(0,classes):


        ##determinant
        inver = inv(np.identity(x.shape[0]) * sigma[c])
        determinant= det(inver)

        part_one_equation = mt.pow(2*mt.pi, -x.shape[0]/2)*mt.pow(determinant,0.5)

        ## values for exp calculation
        value1_exp = ((x -mean[c]).T)
        value1_exp = np.dot(value1_exp, inver)
      

        value2_exp = (x-mean[c])

        ##calc  exp
        exp = np.exp(-0.5*(np.dot(value1_exp, value2_exp)))

        #Result conditional x priori

        rest = part_one_equation*exp

        densities.append(rest*priori[c])
        

    return densities

In [112]:
nb = GaussianNB()
nb.fit(X_test_mfeatFou, y_test_mefeatFou);


priori = nb.class_prior_
sigma = nb.sigma_
mean = nb.theta_
classes = nb.classes_
x = X_test_mfeat

densities_mfeatFou= []
for x in X_test_mfeatFou:
    densities_mfeatFou.append(calc_density(x, priori, mean, sigma, 10))

In [113]:
densities_mfeatFou[0]

[7.017156819282751e-189,
 1.58020801148441e-63,
 3.987019668031554e-90,
 3.839342617918677e-52,
 2.996101557931017e-50,
 3.1107940888017266e-58,
 2.71210508855404e-41,
 9.597647543343547e-68,
 1.957062765836054e-95,
 4.3789518023229234e-41]

In [115]:
def posteriori():
    pass